# 🧪  Recolha e Limpeza

In [196]:
# Importar bibliotecas necessárias
# !pip install pandas
import pandas as pd
from geopy.geocoders import Nominatim
import numpy as np
import os
import openmeteo_requests
import requests_cache
from matplotlib.testing.compare import compare_images
from retry_requests import retry
from datetime import datetime
import requests
import time

In [197]:
# Abrir o dataset
df = pd.read_parquet('datasets/consumos_horario_codigo_postal.parquet')

In [ ]:
# Confirmar a leitura correta das colunas
df.columns

In [ ]:
# Verificar tipos de dados
df.dtypes  # mostra o tipo de cada coluna

In [ ]:
# Verificar n.º de registos
df.shape  # mostra (número de linhas, número de colunas)

In [ ]:
# Visualizar primeiras linhas
df.head()  # primeiros 5 registos

In [ ]:
# Visualizar últimas linhas
df.tail()  # últimos 5 registos

# Engenharia de features
| Feature   | Para que serve?                                   |
|-----------|---------------------------------------------------|
| Day       | Captura ciclos diários                            |
| Month     | Captura sazonalidade mensal                       |
| IsWeekend | 1 se for sábado ou domingo, senão 0               |
| TimeOfDay | Útil porque o consumo e diferente ao longo do dia |
| Season    | 	Captura efeitos climáticos ou sazonais amplos    |

In [198]:
#  Criar a coluna DateTime
df['DateTime'] = pd.to_datetime(df['dt_consumo'].astype(str) + ' ' + df['hr_consumo'], format='%Y-%m-%d %H:%M')

#  Criar a coluna Date
df['Date'] = df['dt_consumo']

# Extrair a hora como número inteiro (ex: 23 de '23:00')
df['Hour'] = pd.to_datetime(df['hr_consumo'], format='%H:%M').dt.hour

#  Criar a coluna ZipCode
df['ZipCode'] = df['codigo_postal']

#  Criar a coluna ActiveEnergy(kWh)
df['ActiveEnergy(kWh)'] = df['consumo']

#  Criar a coluna Day
df['Day'] = df['DateTime'].dt.day

#  Criar a coluna Month
df['Month'] = df['DateTime'].dt.month

#  Criar a coluna Year
df['Year'] = df['DateTime'].dt.year

#  Criar a coluna IsWeekend
df['IsWeekend'] = df['dia_semana'].isin(['Sábado', 'Domingo']).astype(int)

#  Criar a coluna TimeOfDay
def classificar_periodo(hora):
    if 0 <= hora <= 6:
        return 'Noite'
    elif 7 <= hora <= 11:
        return 'Manhã'
    elif 12 <= hora <= 18:
        return 'Tarde'
    else:
        return 'Noite'

df['TimeOfDay'] = df['Hour'].apply(classificar_periodo)

#  Criar a coluna DayOfTheWeek
df['DayOfTheWeek'] = df['dia_semana']

#  Criar a coluna Season
def get_season(month):
    if month in [12, 1, 2]:
        return 'Inverno'
    elif month in [3, 4, 5]:
        return 'Primavera'
    elif month in [6, 7, 8]:
        return 'Verão'
    else:
        return 'Outono'

df['Season'] = df['Month'].apply(get_season)

# Eliminar a coluna Date/Time
df.drop(columns=['datahora', 'dt_consumo', 'hr_consumo', 'codigo_postal', 'consumo', 'dia_semana'], inplace=True)

df.head()

,DateTime,Date,Hour,ZipCode,ActiveEnergy(kWh),Day,Month,Year,IsWeekend,TimeOfDay,DayOfTheWeek,Season
0,2023-03-19 00:00:00,2023-03-19,0,2025,4596.739709,19,3,2023,1,Noite,Domingo,Primavera
1,2023-09-23 12:00:00,2023-09-23,12,4405,14711.438243,23,9,2023,1,Tarde,Sábado,Outono
2,2023-02-15 05:00:00,2023-02-15,5,1600,21440.512557,15,2,2023,0,Noite,Quarta,Inverno
3,2023-02-05 11:00:00,2023-02-05,11,3030,16974.037997,5,2,2023,1,Manhã,Domingo,Inverno
4,2023-02-13 09:00:00,2023-02-13,9,4720,7247.291623,13,2,2023,0,Manhã,Segunda,Inverno


In [ ]:
 # Tarefa: Anotar problemas visíveis
df.dtypes  #  Verifica se os tipos de dados fazem sentido

In [ ]:
# Verifica valores ausentes ou duplicados
df.isnull().sum()
df.duplicated().sum()

In [199]:
# ordenar os dados pela coluna Datetime
df = df.sort_values(by='DateTime').reset_index(drop=True)

In [ ]:
# Verifica se está ordenado corretamente
df.head()  # Ver primeiros valores

In [ ]:
df.tail()  # Ver últimos valores

# Adicionar ao dataset, APIs referentes a temperatura e a densidade populacional

In [200]:
# Dividir o dataset por varios datasets, por codigo postal
for cp in df['ZipCode'].unique(): # Para cada código postal único no dataset
    df_cp = df[df['ZipCode'] == cp].copy()
    df_cp.sort_values('DateTime', inplace=True)  # Ordenar por DateTime (por segurança)

    # Guardar como ficheiro parquet
    caminho_ficheiro = f"./datasets/datasetsPorCP/consumo_{cp}.parquet"
    df_cp.to_parquet(caminho_ficheiro, index=False)
    print(f"Ficheiro guardado: {caminho_ficheiro} ({len(df_cp)} registos)")

Ficheiro guardado: ./datasets/datasetsPorCP/consumo_5000.parquet (8016 registos)
Ficheiro guardado: ./datasets/datasetsPorCP/consumo_2450.parquet (8016 registos)
Ficheiro guardado: ./datasets/datasetsPorCP/consumo_5230.parquet (8016 registos)
Ficheiro guardado: ./datasets/datasetsPorCP/consumo_3620.parquet (8016 registos)
Ficheiro guardado: ./datasets/datasetsPorCP/consumo_2800.parquet (8016 registos)
Ficheiro guardado: ./datasets/datasetsPorCP/consumo_4780.parquet (8016 registos)
Ficheiro guardado: ./datasets/datasetsPorCP/consumo_2410.parquet (8016 registos)
Ficheiro guardado: ./datasets/datasetsPorCP/consumo_4585.parquet (8016 registos)
Ficheiro guardado: ./datasets/datasetsPorCP/consumo_2380.parquet (8016 registos)
Ficheiro guardado: ./datasets/datasetsPorCP/consumo_2985.parquet (8016 registos)
Ficheiro guardado: ./datasets/datasetsPorCP/consumo_5350.parquet (8016 registos)
Ficheiro guardado: ./datasets/datasetsPorCP/consumo_2480.parquet (8016 registos)
Ficheiro guardado: ./dataset

In [201]:
pasta = "./datasets/datasetsPorCP" # Caminho da pasta com os ficheiros parquet
ficheiros = [f for f in os.listdir(pasta) if f.endswith(".parquet")] # Listar todos os ficheiros .parquet da pasta

# Guardar coordenadas em cache para não pedir várias vezes
coordenadas_por_distrito = {}
coordenadas_cache = {}

# Mapeamento simples de prefixos de CPs para distritos
prefixo_para_distrito = {
    range(1000, 2000): "Lisboa",
    range(2000, 2100): "Santarém",
    range(2100, 2300): "Lisboa",
    range(2300, 2400): "Santarém",
    range(2400, 2500): "Leiria",
    range(2500, 2700): "Lisboa",
    range(2700, 2800): "Lisboa",
    range(2800, 3000): "Setúbal",
    range(3000, 3100): "Coimbra",
    range(3100, 3300): "Leiria",
    range(3300, 3500): "Castelo Branco",
    range(3500, 3600): "Viseu",
    range(3600, 3700): "Viseu",
    range(3700, 3800): "Aveiro",
    range(3800, 3900): "Aveiro",
    range(3900, 4000): "Viseu",
    range(4000, 4200): "Porto",
    range(4200, 4400): "Porto",
    range(4400, 4500): "Vila Nova de Gaia",
    range(4500, 4600): "Penafiel",
    range(4600, 4700): "Amarante",
    range(4700, 4900): "Braga",
    range(4900, 5000): "Viana do Castelo",
    range(5000, 5100): "Vila Real",
    range(5100, 5400): "Bragança",
    range(5400, 5500): "Chaves",
    range(5500, 6000): "Bragança",
    range(6000, 6100): "Castelo Branco",
    range(6100, 6200): "Castelo Branco",
    range(6200, 6300): "Covilhã",
    range(6300, 6400): "Guarda",
    range(6400, 6500): "Guarda",
    range(7000, 7100): "Évora",
    range(7100, 7200): "Évora",
    range(7200, 7300): "Évora",
    range(7300, 7400): "Portalegre",
    range(7400, 7500): "Portalegre",
    range(7500, 7600): "Setúbal",
    range(7600, 7700): "Beja",
    range(7700, 7800): "Beja",
    range(7800, 7900): "Beja",
    range(7900, 8000): "Beja",
    range(8000, 8100): "Faro",
    range(8100, 8200): "Faro",
    range(8200, 8300): "Faro",
    range(8300, 8400): "Faro",
    range(8400, 8500): "Faro",
    range(8500, 8600): "Faro",
    range(8600, 8700): "Faro",
    range(8700, 8800): "Faro",
    range(8800, 8900): "Faro",
    range(8900, 9000): "Faro",
    range(9000, 9100): "Madeira",
    range(9100, 9200): "Madeira",
    range(9500, 9600): "Açores",
    range(9600, 9700): "Açores",
    range(9700, 9800): "Açores",
}

In [202]:
# Função auxiliar para obter distrito a partir do código postal
def obter_distrito(codigoPostal):
    try:
        prefixo = int(str(codigoPostal)[:4])
    except ValueError:
        return None  # Código postal inválido, como "OUTROS"

    for intervalo, distrito in prefixo_para_distrito.items():
        if prefixo in intervalo:
            return distrito
    return None  # Prefixo não mapeado

In [204]:
# Função para obter as coordenadas a partir do codigo postal
def obter_coordenadas(codigoPostal):
 # Ignora CPs que não pertençam a nenhum distrito conhecido
    # Ignora casos inválidos como "OUTROS"
    if not str(codigoPostal).isdigit():
        print(f"⚠️ A ignorar código postal inválido: {codigoPostal}")
        return None, None

    distrito = obter_distrito(codigoPostal)     # Verifica se já existem coordenadas em cache por distrito

    if distrito is None:
        print(f"⚠️ Código postal {codigoPostal} não está associado a nenhum distrito conhecido.")
        return None, None

    if distrito in coordenadas_por_distrito:
        return coordenadas_por_distrito[distrito]

    if codigoPostal in coordenadas_cache:
        return coordenadas_cache[codigoPostal]

    # Tenta obter coordenadas usando um CP representativo do distrito
    sufixos = [f"{i:03d}" for i in range(1, 1000, 2)]
    for sufixo in sufixos:
        url = f"https://www.cttcodigopostal.pt/api/v1/3d3ee9c11bb94d1ab881ff6133f48257/{codigoPostal}-{sufixo}"
        try:
            response = requests.get(url, timeout=5)
            response.raise_for_status()
            data = response.json()
            if data and isinstance(data, list):
                lat = float(data[0]["latitude"])
                lon = float(data[0]["longitude"])
                coordenadas_por_distrito[distrito] = (lat, lon)
                return lat, lon
        except requests.exceptions.RequestException:
            continue
        except Exception as e:
            print(f"⚠️ Erro inesperado em {codigoPostal}-{sufixo}: {e}")
            break

    print(f"❌ Falha: {codigoPostal} ({distrito}) sem coordenadas válidas")
    return None, None

In [205]:
# Função para obter a temperatura a partir das coordenadas
def obter_temperatura(lat, lon):
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": "2022-11-01",
        "end_date": "2023-09-30",
        "hourly": "temperature_2m",
        "timezone": "auto"
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        times = pd.to_datetime(data["hourly"]["time"])
        temps = data["hourly"]["temperature_2m"]
        df_temp = pd.DataFrame({"DateTime": times, "Temperature": temps})
        return df_temp
    except Exception as e:
        print(f"❌ Erro ao obter temperaturas: {e}")
        return pd.DataFrame(columns=["DateTime", "Temperature"])

In [ ]:
# Funcao para obter o concelho do codigo postal
def obter_concelho(codigoPostal):



    return concelho

In [ ]:
# Funcao para obter a populacao do concelho do codigo postal
def obter_populacao(codigoPostal):

    populacao = 0

    return populacao

In [ ]:
# funcao para calcular a densidade populacional
def calcular_densidade_populacional(populacao, codigoPostal):

    area = 0.0 # area do concelho do codigo postal

    return populacao / area

In [206]:
# Na pasta datasetsPorCP, abrir cada dataset
for ficheiro in ficheiros:
    caminho_ficheiro = os.path.join(pasta, ficheiro)
    # print(f"\n📂 Abrindo ficheiro: {ficheiro}")
    df = pd.read_parquet(caminho_ficheiro)
    df = df.sort_values("DateTime")

    codigoPostal = str(df["ZipCode"].iloc[0])
    lat, lon = obter_coordenadas(codigoPostal)

    if lat is None or lon is None:
        print(f"⚠️ A saltar {codigoPostal} (sem coordenadas)")
        continue

    df_temp = obter_temperatura(lat, lon)

    if df_temp.empty:
        print(f"⚠️ Sem dados de temperatura para {codigoPostal}")
        continue

    # 🔄 Garantir formatos compatíveis
    df["DateTime"] = pd.to_datetime(df["DateTime"]).dt.floor("H")
    df_temp["DateTime"] = pd.to_datetime(df_temp["DateTime"]).dt.floor("H")

    # 🔗 Juntar pelas datas
    df = pd.merge(df, df_temp, on="DateTime", how="left")


    # densidade populacional
    concelho = obter_concelho(codigoPostal) # obter concelho

    if lat is None or lon is None:
        print(f"⚠️ A saltar {codigoPostal} (sem concelho)")
        continue

    df_temp_densidade_populacional = calcular_densidade_populacional(concelho)

    if df_temp_densidade_populacional.empty:
        print(f"⚠️ Sem dados de densidade populacional para {codigoPostal}")
        continue

    # ? 🔄 Garantir formatos compatíveis (se vai ser intervalo 1dia, ou 1mes, ...)


     # ? 🔗 Juntar pelas datas





    # 💾 Guardar ficheiro
    df.to_parquet(caminho_ficheiro, index=False)
    print(f"✅ Ficheiro atualizado: {ficheiro}")


📂 Abrindo ficheiro: consumo_1000.parquet
✅ Ficheiro atualizado: consumo_1000.parquet

📂 Abrindo ficheiro: consumo_1050.parquet
✅ Ficheiro atualizado: consumo_1050.parquet

📂 Abrindo ficheiro: consumo_1070.parquet
✅ Ficheiro atualizado: consumo_1070.parquet

📂 Abrindo ficheiro: consumo_1100.parquet
✅ Ficheiro atualizado: consumo_1100.parquet

📂 Abrindo ficheiro: consumo_1150.parquet
✅ Ficheiro atualizado: consumo_1150.parquet

📂 Abrindo ficheiro: consumo_1170.parquet
✅ Ficheiro atualizado: consumo_1170.parquet

📂 Abrindo ficheiro: consumo_1200.parquet
✅ Ficheiro atualizado: consumo_1200.parquet

📂 Abrindo ficheiro: consumo_1250.parquet
✅ Ficheiro atualizado: consumo_1250.parquet

📂 Abrindo ficheiro: consumo_1300.parquet
✅ Ficheiro atualizado: consumo_1300.parquet

📂 Abrindo ficheiro: consumo_1350.parquet
✅ Ficheiro atualizado: consumo_1350.parquet

📂 Abrindo ficheiro: consumo_1400.parquet
✅ Ficheiro atualizado: consumo_1400.parquet

📂 Abrindo ficheiro: consumo_1495.parquet
✅ Ficheiro a

In [ ]:
# Unir todos os datasets a apenas um

In [ ]:
# Guardar versão limpa do dataset como .parquet
df = df[df['ActiveEnergy(kWh)'] >= 0]
df.to_parquet('datasets/consumo_eredes_limpo.parquet', index=False)

### Dataset Limpo
- Nome: `consumo_eredes_limpo.parquet`
- Colunas mantidas: `Hour`, `Active Energy (kWh)`, `Day of the Week`
- Colunas criadas: `DateTime`, `Date`, `Day`, `Month`, `Year`, `IsWeekend`, `TimeOfDay`, `Season`, `Temperature`
- `DateTime` ordenado cronologicamente